In [ ]:

!pip install boto3

In [ ]:
import boto3
lambda_client = boto3.client("lambda")
s3_client = boto3.client("s3")

In [ ]:
def s3_bucket_keys(s3_client, bucket_name, bucket_prefix):
    """Generator for listing S3 bucket keys matching prefix"""

    kwargs = {'Bucket': bucket_name, 'Prefix': bucket_prefix}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            yield obj['Key']

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [ ]:
bucket_name =''
bucket_prefix = 'twitter/data/2023/03/'

keys = []

for key in s3_bucket_keys(s3_client=s3_client, bucket_name=bucket_name, bucket_prefix=bucket_prefix):
    keys.append(key)

print(keys)

In [ ]:
import json

for key in keys:

  payload = json.dumps({
    "detail": {
      "requestParameters": {
        "bucketName": bucket_name,
        "key": key
      }
    }
  })

  print(payload)
  response = lambda_client.invoke(
      FunctionName='',
      InvocationType='RequestResponse',
      Payload=payload
  )

  print(response)